In [1]:
import os
import time
import base64
import pandas as pd
from PIL import Image
from io import BytesIO
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options


In [2]:


class ImageDownloader:
    """
    A class to download images from Google Images.

    Parameters:
        path (str): The path to store the downloaded images and CSV file.
        srch (str): The XPath for the search box on the Google Images page.
        second_image (str): The XPath for the second image on the preview page.
        keyword (str): The keyword to search for.
        num_images (int): The number of images to download.
    """

    def __init__(self, path, keyword, num_images):
        self.path = path
        self.keyword = keyword
        self.num_images = num_images
        srch_chrome = '//*[@id="APjFqb"]'
        srch_safari = '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]'
        second_image = '//*[@id="Sva75c"]/div[2]/div/div[2]/div[2]/div[2]/c-wiz/div/div/div/div[3]/div[1]/a/img[1]' #This should not be changed
        self.second_image = second_image



        #Tries to run on chrome if encounters an error try safari
        try:
            
            self.driver= webdriver.Chrome()
            self.srch = srch_chrome
        except:
            self.driver= webdriver.Safari()
            self.srch = srch_safari
        

    def create_directory(self):
        """
        Create the directory to store the downloaded images and CSV file.
        """
        os.makedirs(self.path, exist_ok=True)

    def download_images(self):
        """
        Download images based on the specified parameters.
        """
        
        self.driver.get("http://www.google.com") #Go to google.com
        
        
        
        search_box = self.driver.find_element(By.XPATH, self.srch)
        search_box.send_keys(self.keyword)
        search_box.send_keys(Keys.RETURN) # In the search bar type the keyword

        time.sleep(2)
        
        #If the page is in any other language change it to english
        try:
            images_link = self.driver.find_element(By.LINK_TEXT, "Images") # Go to the Images part of google
        except:
            language_tool = self.driver.find_element(By.XPATH, '//*[@id="Rzn5id"]/div/a[2]') 
            language_tool.click()
            images_link = self.driver.find_element(By.LINK_TEXT, "Images") # Go to the Images part of google

        
        images_link.click()
        time.sleep(2)
        image_elements = self.driver.find_elements(By.CSS_SELECTOR, '.rg_i') #Get all items that are images

        image_info = []
        for i, element in enumerate(image_elements):
            if i == self.num_images: # check the number of images if we reached the limit of predefined number
                break

            element.click()
            time.sleep(2)
            image_forlink = self.driver.find_element(By.XPATH, self.second_image) #Click on the image 
            url = image_forlink.get_attribute('src') #get the url of specific image

            image_url = element.get_attribute('src')

            title = element.get_attribute('alt') #get the title of an image
            if image_url is not None and image_url.startswith('data:image'): 
                encoded_data = image_url.split(',')[1]
                image = Image.open(BytesIO(base64.b64decode(encoded_data))) #transform the link that is available to the format from which we can save the images
                file_path = f'{self.path}/image_{i}.png'
                image.save(file_path)
                print(f"Image {i + 1} downloaded successfully.")
            image_info.append((title, url))

        df = pd.DataFrame(image_info, columns=['Title', 'URL'])
        df.to_csv(f'{self.path}/image_info.csv', mode='w', index=False) # Saving dataframe to csv 

    def quit_driver(self):
        """
        Quit the Selenium driver.
        """
        self.driver.quit()


In [4]:
import tkinter as tk
from tkinter import messagebox

class Application(tk.Frame):
    def __init__(self, master=None):
        super().__init__(master)
        self.master = master
        self.pack()
        self.create_widgets()

    def create_widgets(self):
        self.path_label = tk.Label(self, text="Path")
        self.path_label.pack()
        self.path_entry = tk.Entry(self)
        self.path_entry.pack()

        self.keyword_label = tk.Label(self, text="Keyword")
        self.keyword_label.pack()
        self.keyword_entry = tk.Entry(self)
        self.keyword_entry.pack()

        self.num_images_label = tk.Label(self, text="Number of images")
        self.num_images_label.pack()
        self.num_images_entry = tk.Entry(self)
        self.num_images_entry.pack()

        self.run_button = tk.Button(self)
        self.run_button["text"] = "Download Images"
        self.run_button["command"] = self.download_images
        self.run_button.pack()

    def download_images(self):
        path = self.path_entry.get()
        keyword = self.keyword_entry.get()
        num_images = int(self.num_images_entry.get())

        downloader = ImageDownloader(path, keyword, num_images)
        downloader.create_directory()
        downloader.download_images()
        downloader.quit_driver()
        messagebox.showinfo("Information","Download completed successfully!")

if __name__ == "__main__":
    root = tk.Tk()
    app = Application(master=root)
    app.mainloop()
